In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy import interpolate
import sys
import os
# current_dir = os.path.dirname(os.path.abspath(__file__))

current_dir = os.getcwd()
utils_dir = os.path.abspath(os.path.join(current_dir, '../'))

sys.path.append(utils_dir)
from utils.utils import NORM_COLUMNS, start_date_list, end_date_list


In [ ]:
# ids_pd = pd.read_csv("../utils/117_ids.csv")
# ids = ids_pd['nhdhr_id'].to_list()

# do_raw_path = "../data/raw/lakes/"
# temp_raw_path = '../data/raw/lakes_temp/'
# save_path = '../data/processedByLake/'
# os.makedirs(save_path, exist_ok=True)


# mean_feats = {feature: [] for feature in NORM_COLUMNS}
# std_feats = {feature: [] for feature in NORM_COLUMNS}

# for lake_id in ids:
#     lake_data_path = os.path.join(do_raw_path, f"merged_{lake_id}.csv")
    
#     
#     try:
#         use_do_data = pd.read_csv(lake_data_path, usecols=NORM_COLUMNS)
        
#         
#         mean_by_lake = use_do_data.mean()
#         std_by_lake = use_do_data.std()
        
#         
#         for feature in NORM_COLUMNS:
#             mean_feats[feature].append(mean_by_lake[feature])
#             std_feats[feature].append(std_by_lake[feature])
        
#     except FileNotFoundError:
#         print(f"File for lake {lake_id} not found.")
#         continue


# average_mean_feats = {feature: np.mean(mean_feats[feature]) for feature in NORM_COLUMNS}
# average_std_feats = {feature: np.mean(std_feats[feature]) for feature in NORM_COLUMNS}

# os.makedirs(save_path, exist_ok=True)

# np.save(os.path.join(save_path, 'mean_feats.npy'), average_mean_feats)
# np.save(os.path.join(save_path, 'std_feats.npy'), average_std_feats)

# loaded_mean_feats = np.load(os.path.join(save_path, 'mean_feats.npy'), allow_pickle=True).item()
# loaded_std_feats = np.load(os.path.join(save_path, 'std_feats.npy'), allow_pickle=True).item()

# print("Loaded Mean Features:", loaded_mean_feats)
# print("Loaded Std Features:", loaded_std_feats)

In [ ]:
ids_pd = pd.read_csv("../utils/117_ids.csv")
ids = ids_pd['nhdhr_id'].to_list()

do_raw_path = "../data/raw/lakes/"
temp_raw_path = '../data/raw/lakes_temp/'
save_path = '../data/processedByLake/'
os.makedirs(save_path, exist_ok=True)

all_data = pd.DataFrame()

for lake_id in tqdm(ids):

    lake_data_path = os.path.join(do_raw_path, f"merged_{lake_id}.csv")
    

    try:
        use_do_data = pd.read_csv(lake_data_path, usecols=NORM_COLUMNS)
        all_data = pd.concat([all_data, use_do_data], ignore_index=True)
        
    except FileNotFoundError:
        print(f"File for lake {lake_id} not found.")
        continue

average_mean_feats = all_data.mean().to_dict()
average_std_feats = all_data.std().to_dict()


os.makedirs(save_path, exist_ok=True)


np.save(os.path.join(save_path, 'mean_feats.npy'), average_mean_feats)
np.save(os.path.join(save_path, 'std_feats.npy'), average_std_feats)

loaded_mean_feats = np.load(os.path.join(save_path, 'mean_feats.npy'), allow_pickle=True).item()
loaded_std_feats = np.load(os.path.join(save_path, 'std_feats.npy'), allow_pickle=True).item()

print("Loaded Mean Features:", loaded_mean_feats)
print("Loaded Std Features:", loaded_std_feats)

In [ ]:
temp_raw_path = "../data/raw/lakes_temp/"
processed_save_path = '../data/processedByLake/'


for lake_id in tqdm(ids):
    do_data = pd.read_csv(os.path.join(do_raw_path, f"merged_{lake_id}.csv"))
    norm_do_data = do_data.copy()

    for feature in NORM_COLUMNS:
        mean_value = loaded_mean_feats[feature]
        std_value = loaded_std_feats[feature]
        norm_do_data[feature] = norm_do_data[feature].apply(lambda x: (x - mean_value) / std_value if pd.notna(x) and std_value != 0 else x)
    
    # NaN ----> 0
    norm_do_data[NORM_COLUMNS] = norm_do_data[NORM_COLUMNS].fillna(0)

    temp_data = pd.read_csv(os.path.join(temp_raw_path, f"temperature_{lake_id}.csv"))

    for dataset in ['train', 'val', 'test']:
        start_date = start_date_list[dataset]
        end_date = end_date_list[dataset]

        do_data['datetime'] = pd.to_datetime(do_data['datetime']).dt.tz_localize(None)
        norm_do_data['datetime'] = pd.to_datetime(norm_do_data['datetime']).dt.tz_localize(None)
        temp_data['datetime'] = pd.to_datetime(temp_data['datetime']).dt.tz_localize(None)
        
        train_raw_do_data = do_data[do_data['datetime'].dt.year <= 2011]
        val_raw_do_data = do_data[(do_data['datetime'].dt.year >= 2012) & (do_data['datetime'].dt.year <= 2015)]
        test_raw_do_data = do_data[(do_data['datetime'].dt.year >= 2016) & (do_data['datetime'].dt.year <= 2019)]

        train_norm_do_data = norm_do_data[norm_do_data['datetime'].dt.year <= 2011]
        val_norm_do_data = norm_do_data[(norm_do_data['datetime'].dt.year >= 2012) & (norm_do_data['datetime'].dt.year <= 2015)]
        test_norm_do_data = norm_do_data[(norm_do_data['datetime'].dt.year >= 2016) & (norm_do_data['datetime'].dt.year <= 2019)]


        train_temp_data = temp_data[temp_data['datetime'].dt.year <= 2011]
        val_temp_data = temp_data[(temp_data['datetime'].dt.year >= 2012) & (temp_data['datetime'].dt.year <= 2015)]
        test_temp_data = temp_data[(temp_data['datetime'].dt.year >= 2016) & (temp_data['datetime'].dt.year <= 2019)]

        save_path = f'../data/processedByLake/{lake_id}/'
        os.makedirs(save_path, exist_ok=True)
        train_raw_do_data.to_csv(os.path.join(save_path, 'train_raw_do_data.csv'), index=False)
        val_raw_do_data.to_csv(os.path.join(save_path, 'val_raw_do_data.csv'), index=False)
        test_raw_do_data.to_csv(os.path.join(save_path, 'test_raw_do_data.csv'), index=False)

        train_norm_do_data.to_csv(os.path.join(save_path, 'train_norm_do_data.csv'), index=False)
        val_norm_do_data.to_csv(os.path.join(save_path, 'val_norm_do_data.csv'), index=False)
        test_norm_do_data.to_csv(os.path.join(save_path, 'test_norm_do_data.csv'), index=False)

        train_temp_data.to_csv(os.path.join(save_path, 'train_temp_data.csv'), index=False)
        val_temp_data.to_csv(os.path.join(save_path, 'val_temp_data.csv'), index=False)
        test_temp_data.to_csv(os.path.join(save_path, 'test_temp_data.csv'), index=False)


In [ ]:
import pandas as pd
import json
import os
import numpy as np
import re
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path
base_path = "../data/raw/temperature/"
cfg_path = base_path+"cfg/pb0_config.json"
obs_df = pd.read_csv(base_path+"obs/temperature_observations.csv")
# ids = np.unique(obs_df['site_id'].values)

cfg_f = open(cfg_path)
cfg = json.load(cfg_f)

ct = 0

for nid in ids: #for each new additional lake
    ct += 1
    print("processing lake ",ct,": ",nid)
    nid = str(nid)
    csv = []
    csv.append("depths,areas")
    m = re.search('nhdhr_(.*)', nid) 
    name = m.group(1)

    hs = np.array(cfg[nid]['morphometry']['H'])
    As = cfg[nid]['morphometry']['A']

    #get into posi depth form
    hs = -(np.flip(hs,axis=0) - hs.max())
    As = np.flip(As,axis=0)
    assert hs.shape[0] == As.shape[0]

    #back to str form
    hs = [str(i) for i in hs]
    As = [str(i) for i in As]

    for i in range(len(hs)):
        csv.append(",".join([hs[i], As[i]]))

    
    create_path(f"processedByLake/{nid}/")


    with open("processedByLake/"+nid+"/geometry",'w') as file:
        for line in csv:
            file.write(line)
            file.write('\n')
